In [17]:
!pip install streamlit langchain langchain-community langchain-groq streamlit groq

In [18]:
!pip install scipy

In [19]:
import os
import streamlit as st
import pickle

from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [20]:
groq_api_key = "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"
llm = ChatGroq(api_key=groq_api_key, model_name="llama3-70b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DE98C95210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE98CB4090>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [21]:
def load_and_index_data():
    urls = [
        "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
        "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
    ]

    with st.spinner("📥 Loading data from URLs..."):
        loader = UnstructuredURLLoader(urls=urls)
        data = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.split_documents(data)

    embeddings = HuggingFaceEmbeddings()
    vector_index = FAISS.from_documents(docs, embeddings)

    with open("vector_index.pkl", "wb") as f:
        pickle.dump(vector_index, f)

    return vector_index

In [22]:
if os.path.exists("vector_index.pkl"):
    with open("vector_index.pkl", "rb") as f:
        vector_index = pickle.load(f)
else:
    vector_index = load_and_index_data()

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever(),
    return_source_documents=True
)
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DE98C95210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE98CB4090>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=Secret

In [24]:
docs = vector_index.as_retriever().get_relevant_documents(query)
print("🔎 Retrieved Documents:", len(docs))
for d in docs:
    print(d.page_content[:300], "\n---")


🔎 Retrieved Documents: 4
Trending Topics

Sensex TodayBajaj Finance Share PriceTata Motors Share PriceSpiceJet Share PriceIndian Hotels Share PriceIndiGo Share Price

Wall Street rises as Tesla soars on AI optimism

Tesla (TSLA.O) rallied 10% after Morgan Stanley upgraded the electric car maker to "overweight" from "equal-w 
---
Trending Topics

Mutual FundsGold PriceGold Rate TodaySensex LiveIndia Coal Import

Tata Motors launches Punch iCNG, price starts at Rs 7.1 lakh

The Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switc 
---
The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.

The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.

Tata Motors Passenger Vehicles Ltd Head-Marketing, Vin 
---
Advisory Alert:

Follow Us On:

Stocks: A B C D E F G H I

In [25]:
def get_answer(query, retriever, qa_chain):
    docs = retriever.get_relevant_documents(query)
    print(f"✅ Retrieved {len(docs)} documents for: '{query}'")
    if len(docs) == 0:
        print("❌ No relevant documents found.")
        return {"answer": "No relevant information found.", "sources": ""}
    
    result = qa_chain.invoke({"query": query, "input_documents": docs})
    return result

In [29]:
query = "What is Tiago ongoing price?"
result = get_answer(query, retriever=vector_index.as_retriever(), qa_chain=qa_chain)

print("📄 Answer:", result)
print("🔗 Sources:")
for source in result.get("sources", "").split("\n"):
    print("-", source)

✅ Retrieved 4 documents for: 'What is Tiago ongoing price?'
📄 Answer: {'query': 'What is Tiago ongoing price?', 'input_documents': [Document(id='3a6f762a-6255-4708-b35c-1f9d52b79871', metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}, page_content='The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company\'s CNG line up "appealing, holistic, and stronger than ever".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nDiscover the latest Business News, Budget 2025 News, Sensex, and Nifty updates. Obtain Personal Finance insights, tax queries, and expert opinions on Moneycontrol or download t

In [27]:
print(result)

{'query': "What is Tata's stock trend?", 'input_documents': [Document(id='25857c52-aaaf-4d1b-8206-6413ddeb3c88', metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='Trending Topics\n\nSensex TodayBajaj Finance Share PriceTata Motors Share PriceSpiceJet Share PriceIndian Hotels Share PriceIndiGo Share Price\n\nWall Street rises as Tesla soars on AI optimism\n\nTesla (TSLA.O) rallied 10% after Morgan Stanley upgraded the electric car maker to "overweight" from "equal-weight," saying its Dojo supercomputer could boost the company\'s market value by nearly $600 billion.\n\nReuters\n\nSeptember 12, 2023 / 06:56 IST\n\nWall Street rises as Tesla soars on AI optimism\n\nThe Nasdaq closed sharply higher on Monday as Tesla surged on optimism around artificial intelligence and investors awaited inflation data due later this week.\n\nTesla (TSLA.O) rallied 10% after Morgan Stanley upgraded the elect

In [32]:
import langchain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [33]:
prompt = PromptTemplate.from_template("Qn: {Qn}\nAnswer:")
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\sachi\AppData\Local\Temp\ipykernel_4960\3087775585.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [35]:
query = "what is the on ging price of tiago?"
langchain.debug=True
chain({"Qn": query}, return_only_outputs=True)

[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "Qn": "what is the on ging price of tiago?"
}
[llm/start] [chain:LLMChain > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "Human: Qn: what is the on ging price of tiago?\nAnswer:"
  ]
}
[llm/end] [chain:LLMChain > llm:ChatGroq] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The on-going price of Tata Tiago in India is:\n\n* Base Model (XE): ₹4.85 lakh - ₹5.15 lakh\n* Mid-variant (XM): ₹5.35 lakh - ₹5.65 lakh\n* Top-variant (XT): ₹5.85 lakh - ₹6.15 lakh\n* CNG variant: ₹6.35 lakh - ₹6.65 lakh\n\nPlease note that prices may vary depending on the city, state, and other factors. It's always a good idea to check the exact price with your local dealership or Tata Motors website.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "

{'text': "The on-going price of Tata Tiago in India is:\n\n* Base Model (XE): ₹4.85 lakh - ₹5.15 lakh\n* Mid-variant (XM): ₹5.35 lakh - ₹5.65 lakh\n* Top-variant (XT): ₹5.85 lakh - ₹6.15 lakh\n* CNG variant: ₹6.35 lakh - ₹6.65 lakh\n\nPlease note that prices may vary depending on the city, state, and other factors. It's always a good idea to check the exact price with your local dealership or Tata Motors website."}